## Observer

An observer is an object that wishes to be informed about events happening in the system. The entity generating the events is an observable.

In [18]:
class Event(list):
    def __call__(self, *args, **kwargs):
        for item in self:
            item(*args, **kwargs)

class Person:
    def __init__(self, name, address):
        self.name = name
        self.address = address
        self.falls_ill = Event()
        
    def catch_a_cold(self):
        self.falls_ill(self.name, self.address)

def call_doctor(name, address):
    print(f"{name} needs a doctor at {address}")
        

p = Person('John', '123 London Road')

p.falls_ill.append(lambda *args: print(f"{args[0]} wishes a good luck!"))
p.falls_ill.append(call_doctor)
p.catch_a_cold()
print()
p.falls_ill.remove(call_doctor)
p.catch_a_cold()

John wishes a good luck!
John needs a doctor at 123 London Road

John wishes a good luck!


### Property Observers

In [4]:
class Event(list):
    def __call__(self, *args, **kwargs):
        for item in self:
            item(*args, **kwargs)
        
        
class PropertyObservable:
    def __init__(self):
        self.property_changed = Event()
        
        
class Person(PropertyObservable):
    def __init__(self, age=0):
        super().__init__()
        self._age = age
    
    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, value):  # When setting age -> calling Event __call__
        if self._age == value:
            return
        self._age = value
        self.property_changed('age', value)
        
        
class TrafficAuthority:
    def __init__(self, person):
        self.person = person
        person.property_changed.append(  # Appended person_changed
            self.person_changed
        )
    
    def person_changed(self, name, value):
        if name == "age":
            if value < 16:
                print("Sorry, you still cannot drive")
            else:
                print("Okay, you can drive now")
                self.person.property_changed.remove(
                    self.person_changed
                )

p = Person()
ta = TrafficAuthority(p)
for age in range(14, 20):
    print(f"Setting age to {age}")
    p.age = age
    print()

Setting age to 14
Sorry, you still cannot drive

Setting age to 15
Sorry, you still cannot drive

Setting age to 16
Okay, you can drive now

Setting age to 17

Setting age to 18

Setting age to 19



### Property Dependencies

In [7]:
class Event(list):
    def __call__(self, *args, **kwargs):
        for item in self:
            item(*args, **kwargs)
        
        
class PropertyObservable:
    def __init__(self):
        self.property_changed = Event()
        
        
class Person(PropertyObservable):
    def __init__(self, age=0):
        super().__init__()
        self._age = age
    
    @property
    def can_vote(self):
        return self._age >= 18
    
    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, value):  # When setting age -> calling Event __call__
        if self._age == value:
            return
        
        old_can_vote = self.can_vote
        
        self._age = value
        self.property_changed('age', value)
        
        if old_can_vote != self.can_vote:
            self.property_changed('can_vote', self.can_vote)
        
        
class TrafficAuthority:
    def __init__(self, person):
        self.person = person
        person.property_changed.append(  # Appended person_changed
            self.person_changed
        )
    
    def person_changed(self, name, value):
        if name == "age":
            if value < 16:
                print("Sorry, you still cannot drive")
            else:
                print("Okay, you can drive now")
                self.person.property_changed.remove(
                    self.person_changed
                )
                
                
def person_changed(name, value):
    if name == "can_vote":
        print(f"Voting ability changed to {value}")
                
p = Person()
p.property_changed.append(
    person_changed
)

for age in range(16, 21):
    print(f"Changing age to {age}")
    p.age = age

Changing age to 16
Changing age to 17
Changing age to 18
Voting ability changed to True
Changing age to 19
Changing age to 20
